# Advanced Regression

In this lesson we will learn a number of techniques for advanced regression. Most of them are already known algorithms from classification that can be reused for regression task with simple or serious changes. 

### Bayesian approach to regression


But first, let's start with a little improvement to understanding the structure of standard linear regression with regularization. We know that regularization can help us with over-fitting and controlling the regression coefficients. However, there are different explanations of how this happens, and one of them is quite simple, but [demands some knowledge of statistics and distribution](http://bjlkeng.github.io/posts/probabilistic-interpretation-of-regularization/). 

<span style="color:red">Exercise:</span> study the link in parallel with this notebook.

To be short, we use maximal likelihood formulation of regression and assumptions about residuals (that they are normally distributed). However, now we use the posterior probability according to Bayes theorem:
$$
\begin{align*}
P(\theta | y) &= \frac{P(y | \theta) P(\theta)}{P(y)} \\
\text{posterior} &= \frac{\text{likelihood} \cdot \text{prior}}{\text{evidence}}
\tag{5}
\end{align*}
$$
Instead of trying to maximize the probability of our data given the parameters, we want to maximize the probability of chosen parameters, given the data. Because the data is known, $P(y)$ is always constant, so we do not need it in maximization problem, so only thing left is $P(\theta)$. This is called prior and we can choose it however we want. Let's make one more assumption and say that our coefficients are distributed accordingly to normal distribution with zero mean:
![](https://upload.wikimedia.org/wikipedia/commons/thumb/7/74/Normal_Distribution_PDF.svg/1920px-Normal_Distribution_PDF.svg.png)
Basically, we are saying that our coefficients are distributed around zero, with different density. if $\sigma$ is small, they are closely packed around 0, otherwise there is a room for them to change around a significant interval. 
Now we know all the components of the function we are trying to maximize. Going through mathematical transformations, presented by the link, in the end we obtain the following expression:
$$
\arg\min_{\bf \beta} \big[ \sum_{i=1}^{n} (y_i-(\beta_0 + \beta_1 x_{i,1} + ... + \beta_p x_{i,p}))^2 + \lambda \sum_{j=0}^{p} \beta_j^2 \big]
$$
Which is actually Ridge regression with $\lambda = \frac{\sigma^2}{\theta^2}$, where $\sigma$ is standard deviation for residuals and $\theta$ is standard deviation for coefficients. So, our conclusions:
- Using posterior regression formulation with normal prior we obtain exactly ridge regression
- Increasing $\lambda$ leads to lesser coefficients and stricter fit, here is why:
    - If we increase $\sigma$ or the deviation of data points from our regression line, $\lambda$ is increased and vice versa: the bigger the parameter of regularization, the lesser our regression fit to data points and vice versa (simple linear regression at $\lambda=0$ is the best fit as it should be (but quite possibly overfitted!))
    - If we increase $\theta$ or the spread of coefficients, allowing them to be further from zero, $\lambda$ decreases, so we relax the condition on coefficients, which leads to better fit. Otherwise, the condition is more strict, $\lambda$ is increased, we have less room for coefficients to go around.
Lasso regression can be obtained the same way, but using Laplace distribution as prior (which is similar to normal, but uses absolute value instead of square in the density formula)

Note: actually knowing $\lambda$, we can calculate $\sigma$ after fit and $\theta$ using this formula: $\lambda = \frac{\sigma^2}{\theta^2}$. Now we know the exact distribution of coefficients and can exactly calculate the significance of each coefficient.

### Other advanced regression methods

Now, let's talk about other regression methods. Why do we need them?
- Linear regression is "linear", there are no feature engineering (at least automatic, of course you can do it yourself, or use polynomial regression)
- There is a lot of assumptions, under which using linear regression is "correct", including:
    - Linearity
    - Residuals have normal distribution and independent
    - Homoscedacity
So, If our data has some complex structure, linear regression is definitely not the way to go. Assuming, we use polynomial regression, we can get the following results for the example:

![](https://cdn-images-1.medium.com/max/1000/0*puia9Ua6HacHTj7j.)

On the picture (a) we use low degree of a polynomial, so in the region with high volatility, the model behaves poorly. Otherwise, using greater degrees, these regions are covered, but "constant" region is fluctuating all over the place. So, usual linear regression can not cover data with different distributions across the domain.

However, it is possible for known classification methods, such as KNN, Decision Trees and others. It would be great if we could use them for regression... we actually can! And for most algorithms, changes are elementary. On the next picture you can see how decision tree solves regression for this example.

![Boosting Regression](https://cdn-images-1.medium.com/max/1600/0*mHWDQgtSduq7rzj9.)

<span style="color:red">Exercise:</span> Think with students about how we can transform/modify
- KNN
- Decision Tree
- Random Forest
- AdaBoost
- SVM
for them to become regression models.

First, K Nearest Neighbors algorithm. It is intuitive in its nature for classification, and for regression actually as well. If for classification, the prediction is based on majority ruling of nearest neighbors, then for regression it uses an approach of **averaging** the results of the neighbors. It also can be done using weights for neighbors, based on their distance from the ponit in question.

[Read about the boston dataset](https://scikit-learn.org/stable/datasets/index.html#boston-dataset)

In [ ]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
data, target = load_boston(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(data, target)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score

In [ ]:
knr = KNeighborsRegressor(n_neighbors=3, weights='distance')
knr.fit(X_train, y_train)
y_pred = knr.predict(X_test)
print(f'R2 score: {r2_score(y_pred, y_test)}')

R2 score: 0.5796131998767747


Next on our list are Decision Trees. It's actually also simple to also make regression model from this: Every leaf is now considered as some kind of neighborhood, and prediction for an object is an average of other objects contained in the leaf, that we got to making prediction. When building the decision tree, instead of information criterion we could use deviation criterions, such as standard deviation. The parameters of this Regression Decision Tree relating to branching are the same.
Total number of possible prediction values is equal to the number of leaves in the tree. It could be seen on the picture above, which consists of horizontal (areas with the same prediction values) and vertical lines (connecting areas)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score

In [ ]:
dtr = DecisionTreeRegressor(max_depth=5, min_samples_split=10)
dtr.fit(X_train, y_train)
y_pred = dtr.predict(X_test)
print(f'R2 score: {r2_score(y_pred, y_test)}')

R2 score: 0.7662564839903987


Using previous models, it is easy to predict how we change Random Forest algorithm for regression - we are building trees according to previously described Decision Trees with limited number of features for every tree; after that we averafe the results from every tree to obtain the answer.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

In [ ]:
rfr = RandomForestRegressor(n_estimators=25, max_depth=5, min_samples_split=10)
rfr.fit(X_train, y_train)
y_pred = rfr.predict(X_test)
print(f'R2 score: {r2_score(y_pred, y_test)}')

R2 score: 0.8284313283548286


Boosting algorithm is actually more dependent on the type of loss function than on the problem itself. So, the only hard thing is to find loss function for regression problem instead of classification. Full algorithm is described in [this](https://www.researchgate.net/profile/Harris_Drucker/publication/2424244_Improving_Regressors_Using_Boosting_Techniques/links/0deec51ae736538cec000000/Improving-Regressors-Using-Boosting-Techniques.pdf) article.

In [ ]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import r2_score

In [ ]:
abr = AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=5, min_samples_split=10), n_estimators=25)
abr.fit(X_train, y_train)
y_pred = abr.predict(X_test)
print(f'R2 score: {r2_score(y_pred, y_test)}')

R2 score: 0.8473495230359775


SVM is the hardest algorithm in the current list. The main idea is to reverse the classification idea: instead of separation strip between correctly classified points/areas we create the strip for the points that considered to be correct - all points that are outside of this strip are considered as errors. The mathematical formulation can be found [here](https://www.mathworks.com/help/stats/understanding-support-vector-machine-regression.html), but to be short, we are modelling two SVM lines, parallel to each other, trying to make the distance between as small (not big!) as possible. There is a way to introduce C-SVR as well (margin of error outside of two SVM lines), the only difference is that these margins may be different from each side of the strip. And, kernel trick works too. So, the algorithm is still used to transform the space and engineer new features.

In [ ]:
from sklearn.svm import SVR
from sklearn.metrics import r2_score

In [ ]:
svr = SVR(C=500000, kernel='rbf')
svr.fit(X_train, y_train)
y_pred = svr.predict(X_test)
print(f'R2 score: {r2_score(y_pred, y_test)}')

R2 score: 0.8706805024535895


<span style="color:red">Practical exercise:</span> Use Polynomial, Ridge or Lasso linear regressions for the same dataset, trying to get better r2 score.